In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

file = "/Users/erickxu/Desktop/CSP/Python/data_sets/brand_posts.xlsx"

df = pd.read_excel(io = file, header = 0)
df.head(n = 5)

,created_time,message,type,story,likes_count,comments_count,shares_count,love_count,haha_count,wow_count,sad_count,angry_count,comments
0,2012-06-20 15:12:38,"Ummm, yum! What would be in your ULTIMATE naug...",photo,Australis Cosmetics with Evalyn McGregor-Koch ...,715,47,46,0.0,0.0,0.0,0.0,0.0,that looks awesome Icecream with crushed oreo....
1,2012-06-21 15:09:46,Click 'like' if you worry about that too! <3 <3,photo,NaN,1103,8,80,0.0,0.0,0.0,0.0,0.0,:\ true. i agree I just moved and I am particu...
2,2012-06-22 10:41:36,Seems like some celebs can get away with any h...,photo,NaN,161,12,2,0.0,0.0,0.0,0.0,0.0,blonde her eye colour is different Way better ...
3,2012-06-22 15:37:04,We absolutely adore our bling here at Australi...,photo,NaN,30,0,0,0.0,0.0,0.0,0.0,0.0,NaN
4,2012-06-25 10:26:10,You know you want to... :P,photo,Australis Cosmetics with Evalyn McGregor-Koch ...,117,0,6,0.0,0.0,0.0,0.0,0.0,NaN


In [2]:
# Fill missing values with NA or 0 for str and int/float types respectively
df["story"] = df["story"].fillna("NA")
df["comments"] = df["comments"].fillna("NA")
df["message"] = df["message"].fillna("NA")

df.iloc[:, 7:12] = df.iloc[:, 7:12].fillna(0)

In [3]:
# Makes everything lowercase
df = df.astype(str).apply(lambda col: col.str.lower())

# Select colomns
cols_to_clean = list(df.columns[1:4]) + [df.columns[12]]

# Clean selected columns
df[cols_to_clean] = df[cols_to_clean].map(
    lambda x: re.sub(r'[^A-Za-z\s]', '', x) if isinstance(x, str) else x
)

In [4]:
# Stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


# Clean text of stop words
def clean_text(text):
    
    text = re.sub(r'[^a-zA-Z\s]', "", text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    
    return text

# Apply function
df["message"] = df["message"].apply(clean_text)
df["story"] = df["story"].apply(clean_text)
df["comments"] = df["comments"].apply(clean_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/erickxu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Make sure column is actually in datetime format
df["created_time"] = pd.to_datetime(df["created_time"])

# Get the date
df["date"] = df["created_time"].dt.date

# Get the hour
df["hour"] = df["created_time"].dt.hour

# Drop the original column
df.drop("created_time", axis=1, inplace=True)

# Get list of all columns
cols = list(df.columns)

# Move day and time columns to front
df = df[cols[-2:] + cols[:-2]]


In [6]:
df.head(n = 5)

,date,hour,message,type,story,likes_count,comments_count,shares_count,love_count,haha_count,wow_count,sad_count,angry_count,comments
0,2012-06-20,15,ummm yum would ultimate naughty treat,photo,australis cosmetics evalyn mcgregorkoch others,715,47,46,0.0,0.0,0.0,0.0,0.0,looks awesome icecream crushed oreo milo somet...
1,2012-06-21,15,click like worry,photo,na,1103,8,80,0.0,0.0,0.0,0.0,0.0,true agree moved particually worried first mis...
2,2012-06-22,10,seems like celebs get away hair colour check n...,photo,na,161,12,2,0.0,0.0,0.0,0.0,0.0,blonde eye colour different way better blonde ...
3,2012-06-22,15,absolutely adore bling australis hq nothing ma...,photo,na,30,0,0,0.0,0.0,0.0,0.0,0.0,na
4,2012-06-25,10,know want p,photo,australis cosmetics evalyn mcgregorkoch shadma...,117,0,6,0.0,0.0,0.0,0.0,0.0,na


In [7]:
# Save the DataFrame to an Excel file
df.to_excel("clean_brand_posts_data.xlsx", index=False)
